In [1]:
# importando libs
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import os
from datetime import datetime
from datetime import timedelta
import yfinance as yf
import pickle
from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json
from prophet import plot
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

/home/fabricio/Documentos/Projetos/tcc_predict_crypto/model_predict/.predict/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DAYS_PREDICT = 15

In [3]:
date_today = datetime.today().strftime("%Y-%m-%d")
data_init = "2014-01-01"

df_matic = yf.download("MATIC-USD", data_init, date_today)

df_matic.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-04-28,0.004521,0.004545,0.004317,0.004421,0.004421,8408136
2019-04-29,0.004418,0.004492,0.003471,0.003471,0.003471,8267044
2019-04-30,0.003462,0.005078,0.003458,0.004478,0.004478,25072706
2019-05-01,0.004440,0.004673,0.004071,0.004374,0.004374,10567352
2019-05-02,0.004366,0.004700,0.004096,0.004198,0.004198,6714908


In [4]:
df_matic.reset_index(inplace=True)
df_matic.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
1233,2022-09-12,0.894320,0.939656,0.875008,0.930012,0.930012,676653496
1234,2022-09-13,0.930041,0.940589,0.842449,0.854414,0.854414,708971458
1235,2022-09-14,0.846233,0.867266,0.837794,0.864789,0.864789,414726283
1236,2022-09-15,0.864793,0.873348,0.823307,0.824299,0.824299,459779689
1237,2022-09-16,0.824382,0.830208,0.791151,0.806046,0.806046,362934299


In [5]:
df = df_matic[["Date", "Adj Close"]]
df.rename(columns= { "Date": "ds", "Adj Close": "y" }, inplace=True)

In [6]:
df.tail()

,ds,y
1233,2022-09-12,0.930012
1234,2022-09-13,0.854414
1235,2022-09-14,0.864789
1236,2022-09-15,0.824299
1237,2022-09-16,0.806046


In [7]:
assert df.isnull().sum().sum() == 0
assert df['ds'].isnull().sum() == 0
assert df['y'].isnull().sum() == 0

In [8]:
df.dtypes

ds    datetime64[ns]
y            float64
dtype: object

In [9]:
model = Prophet(seasonality_mode="multiplicative")
model.fit(df)

00:34:34 - cmdstanpy - INFO - Chain [1] start processing
00:34:35 - cmdstanpy - INFO - Chain [1] done processing


In [10]:
df_future = model.make_future_dataframe(periods=DAYS_PREDICT)
df_future.tail()

,ds
1248,2022-09-27
1249,2022-09-28
1250,2022-09-29
1251,2022-09-30
1252,2022-10-01


In [11]:
# predict 
predict = model.predict(df_future)
predict

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2019-04-28,0.015201,-0.180326,0.210264,0.015201,0.015201,-0.167154,-0.167154,-0.167154,-0.001395,-0.001395,-0.001395,-0.165759,-0.165759,-0.165759,0.0,0.0,0.0,0.012660
1,2019-04-29,0.015201,-0.175782,0.203477,0.015201,0.015201,-0.180403,-0.180403,-0.180403,-0.010322,-0.010322,-0.010322,-0.170081,-0.170081,-0.170081,0.0,0.0,0.0,0.012459
2,2019-04-30,0.015201,-0.185905,0.207925,0.015201,0.015201,-0.172779,-0.172779,-0.172779,0.000655,0.000655,0.000655,-0.173434,-0.173434,-0.173434,0.0,0.0,0.0,0.012575
3,2019-05-01,0.015202,-0.188769,0.196439,0.015202,0.015202,-0.165471,-0.165471,-0.165471,0.010326,0.010326,0.010326,-0.175797,-0.175797,-0.175797,0.0,0.0,0.0,0.012686
4,2019-05-02,0.015202,-0.175788,0.203393,0.015202,0.015202,-0.172358,-0.172358,-0.172358,0.004793,0.004793,0.004793,-0.177151,-0.177151,-0.177151,0.0,0.0,0.0,0.012582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248,2022-09-27,0.848760,0.391672,0.782940,0.848535,0.848760,-0.295413,-0.295413,-0.295413,0.000655,0.000655,0.000655,-0.296068,-0.296068,-0.296068,0.0,0.0,0.0,0.598025
1249,2022-09-28,0.846391,0.399292,0.788118,0.845800,0.846391,-0.293443,-0.293443,-0.293443,0.010326,0.010326,0.010326,-0.303769,-0.303769,-0.303769,0.0,0.0,0.0,0.598023
1250,2022-09-29,0.844022,0.384246,0.780991,0.842847,0.844022,-0.305342,-0.305342,-0.305342,0.004793,0.004793,0.004793,-0.310135,-0.310135,-0.310135,0.0,0.0,0.0,0.586307
1251,2022-09-30,0.841653,0.384680,0.782384,0.839955,0.841809,-0.315744,-0.315744,-0.315744,-0.000799,-0.000799,-0.000799,-0.314945,-0.314945,-0.314945,0.0,0.0,0.0,0.575906


In [12]:
predict[["ds", "yhat_lower", "yhat_upper", "yhat"]].tail()

,ds,yhat_lower,yhat_upper,yhat
1248,2022-09-27,0.391672,0.782940,0.598025
1249,2022-09-28,0.399292,0.788118,0.598023
1250,2022-09-29,0.384246,0.780991,0.586307
1251,2022-09-30,0.384680,0.782384,0.575906
1252,2022-10-01,0.359376,0.759017,0.569659


In [13]:
# Salvar modelo
with open('model_matic_usd.pckl', 'wb') as fout:
    pickle.dump(model, fout)